In [51]:
from IPython.display import clear_output
import os.path as path
import pandas as pd
import pickle
import pprint
import PyPDF2 as pdf
import random
import re
import requests
import textract
import uuid
import time

# Config

data_directory is where the input files are expected to be found and where the output files will be written to

In [2]:
data_directory = 'data/'
prefix = 'out_'
mock = False # Mock website connection

Seed random function to get reproducable results:

In [3]:
random.seed(666)

# Helper Functions

In [4]:
def extract_from_page(url, pattern):
    headers = {
        'accept': '*/*',
        'accept-language': 'en,de-DE;q=0.9,de;q=0.8,en-US;q=0.7',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Brave Chrome/83.0.4103.116 Safari/537.36',
    }
    response = requests.get(url, headers=headers)
    if response.status_code > 299:
        print(f'Error extracting from {url}: Status {response.status_code} Reason {response.reason}')
        return None
    html = response.text
    match = re.search(pattern, html)
    if match:
        return match.group(1)
    else:
        # print(f'Error extracting from {url}: No match found {pattern} in {html}')
        return None

In [5]:
def get_name(ticker):
    if mock:
        return "Mocked Company Name"
    # Try barchart
    barchart_result = extract_from_page(f'https://www.barchart.com/stocks/quotes/{ticker}/overview', 
                                        '<span class="symbol">([^<]+)</span>')
    if barchart_result:
        return barchart_result
    
    # Try yahoo
    yahoo_result =  extract_from_page(f'https://finance.yahoo.com/quote/{ticker}', 
                                        '<h1 class="D\(ib\) Fz\(18px\)">([^<]+) \([A-Z]+\)\</h1>')
    if barchart_result:
        return barchart_result
    
    # if still no result try marketwatch (takes around 10s)
    marketwatch_result = extract_from_page(f'https://www.marketwatch.com/investing/stock/{ticker}',
                                           '.*<h1 class="company__name">([^<]+)</h1>.*')
    
    if marketwatch_result:
        return marketwatch_result
    else:
        print(f'Invalid ticker: {ticker}')


In [6]:
extract_from_page(f'https://www.barchart.com/stocks/quotes/TSLA/overview', 
                                        '<span class="symbol">([^<]+)</span>')

'Tesla Inc'

In [7]:
extract_from_page(f'https://finance.yahoo.com/quote/TSLA', 
                                        '<h1 class="D\(ib\) Fz\(18px\)">([^<]+) \([A-Z]+\)\</h1>')

'Tesla, Inc.'

In [8]:
extract_from_page(f'https://www.marketwatch.com/investing/stock/TSLA',
                                           '.*<h1 class="company__name">([^<]+)</h1>.*')

'Tesla Inc.'

In [9]:
def get_exchange_online(ticker):
    if mock:
        return "Mock Exchange"
    
    exchange_alias = {
        'NYSE': 'New York Stock Exchange Inc.'
    }
    # Try barchart
    barchart_result = extract_from_page(f'https://www.barchart.com/stocks/quotes/{ticker}/overview', 
                                        '<span class="symbol-trade-time">\[([^\]]+)\]<\/span>')
    if barchart_result:
        exchange = barchart_result
    else:
        print(f'No exchange found for: {ticker}')
        return
    if exchange in exchange_alias:
        return exchange_alias[exchange]
    else:
        return exchange

In [10]:
get_exchange_online('KO')

'New York Stock Exchange Inc.'

In [11]:
get_exchange_online('TSLA')

'NASDAQ'

In [12]:
def track_progress(function, arguments_list):
    n = len(arguments_list)
    return_values = []
    for i, a in enumerate(arguments_list):
        return_values.append(function(a))
        clear_output(wait=True)
        print( '%.2f' % (100*(i+1)/n), '% done', end='\r',flush=True)
    return return_values

In [13]:
res = track_progress(lambda _: time.sleep(0.01), [i for i in range(100)])

# Type Tables

In [14]:
fi_types = ['stock', 'option', 'swap', 'future', 'forward', 'equity-linked security']

In [15]:
fi_types_df = pd.DataFrame(fi_types, columns = ['mnemonic'])
fi_types_df.index.name = 'fi_type_code'
fi_types_df

,mnemonic
fi_type_code,
0,stock
1,option
2,swap
3,future
4,forward
5,equity-linked security


In [16]:
fi_types_df.to_csv(f'{data_directory}{prefix}fi_type.csv', header=False)

In [17]:
option_styles = ['european', 'american', 'asian']

In [18]:
option_styles_df = pd.DataFrame(option_styles, columns = ['mnemonic'])
option_styles_df.index.name = 'option_style_code'
option_styles_df

,mnemonic
option_style_code,
0,european
1,american
2,asian


In [19]:
option_styles_df.to_csv(f'{data_directory}{prefix}option_style.csv', header=False)

# Trader

In [20]:
in_trader_df = pd.read_csv(data_directory + '15153231.txt', delimiter = '\t' )
in_trader_df.head()

,FIRST,LAST,NICKNAME,EMP-NO,2019 BONUS,2019 BASE,2019 TOTAL COMP,2019 PERFORMANCE,DECISION
0,Madyson,Holden,Admiral,4234,248478,78490,326968,4,keep
1,Rowan,Owens,Amazon,2344,227649,3895,231544,7,keep
2,Bridger,Mcclain,Bean,2122,53720,84662,138382,5,keep
3,Maren,Adkins,Beast,3221,242405,90037,332442,2,FIRE
4,Amira,Colon,Big Nasty,2366,46265,77211,123476,6,keep


Pad employee id with zeroes to allow for more than 9999 employees (high ids indicate that an overflow is close):

In [21]:
in_trader_df['emp_id'] = [str(e).zfill(8) for e in in_trader_df['EMP-NO']]

Rename column to lowercase naming convention and strip nickname of whitespaces

In [22]:
in_trader_df['nickname'] = [n.strip() for n in in_trader_df['NICKNAME']]

Identify duplicate values, keep first occurence

In [23]:
in_trader_df['duplicated'] = in_trader_df.duplicated(subset=['emp_id'])
in_trader_df[in_trader_df['duplicated'] == True]

,FIRST,LAST,NICKNAME,EMP-NO,2019 BONUS,2019 BASE,2019 TOTAL COMP,2019 PERFORMANCE,DECISION,emp_id,nickname,duplicated
11,Sanai,Mccann,Chuckles,4234,65174,40579,105753,6,keep,00004234,Chuckles,True


Remove duplicates and create table ready for import

In [24]:
trader_df = pd.DataFrame(in_trader_df[in_trader_df['duplicated'] == False][['emp_id', 'nickname']])
trader_df.head()

,emp_id,nickname
0,00004234,Admiral
1,00002344,Amazon
2,00002122,Bean
3,00003221,Beast
4,00002366,Big Nasty


In [25]:
trader_df.to_csv(f'{data_directory}{prefix}trader.csv', header=False, index=False)

# Counterparty

In [26]:
in_cp_df = pd.read_csv(data_directory + 'COUNTERPARTY.csv')
in_cp_df.head()

,NAME,ASSETS,PARENT
0,China Industrial and Commercial Bank of China,"4,009.26",-1
1,China China Construction Bank Corporation,"3,400.25",-1
2,China Agricultural Bank of China,"3,235.65",-1
3,China Bank of China,"2,991.90",-1
4,Japan Mitsubishi UFJ Financial Group,"2,784.74",-1


The counterparties in this file are all external as opposed to internal counterparties

In [27]:
in_cp_df['cp_type'] = ['external' for _ in range(len(in_cp_df))]

Generate GUID for counterparties:

In [29]:
in_cp_df['guid'] = [str(uuid.UUID(int=random.getrandbits(128))) for _ in range(len(in_cp_df))]

In [30]:
in_cp_df['name'] = in_cp_df['NAME']

inspecting for duplicates:

In [31]:
in_cp_df['duplicated'] =in_cp_df.duplicated(subset=['name'])
in_cp_df[in_cp_df['duplicated'] == True]

,NAME,ASSETS,PARENT,cp_type,guid,name,duplicated


no duplicates found

In [32]:
cp_df = in_cp_df[['guid', 'cp_type', 'name']]
cp_df.head()

,guid,cp_type,name
0,03a41ee6-8029-1220-4893-6f196f333b8a,external,China Industrial and Commercial Bank of China
1,c762c7fd-8dbd-8ff2-8c38-2cb9cfb54140,external,China China Construction Bank Corporation
2,0c9d7fb2-1f86-3d88-5548-0e0ab65115f8,external,China Agricultural Bank of China
3,fa355d27-62af-b68e-2455-72fed0941885,external,China Bank of China
4,942ef357-e44e-b02b-7795-893da10d59bd,external,Japan Mitsubishi UFJ Financial Group


In [33]:
cp_df.to_csv(f'{data_directory}{prefix}counterparty.csv', header=False, index=False)

# Stock

Collect stocks from all file that are to be loaded

## FIRSTPRICES.pdf

**MANUAL STEP**: Copy content of FIRSTPRICES.pdf to text editor and save as FIRSTPRICES.txt

In [34]:
in_trades_df = pd.read_csv(data_directory + 'FIRSTPRICES.txt', delimiter=' ')
in_trades_df.head()

,TICKER,ACTION,SHARES,LEVERAGE,PRICE
0,META,B,20001,1.0,912.11
1,DHY,B,401,2.1,2.14
2,FAX,B,2440,3.3,3.28
3,DSU,S,99393,4.0,92.11
4,XOM,B,266,1.0,213.22


Check for missing values:

In [35]:
in_trades_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TICKER    15 non-null     object 
 1   ACTION    15 non-null     object 
 2   SHARES    15 non-null     int64  
 3   LEVERAGE  13 non-null     float64
 4   PRICE     15 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 728.0+ bytes


Two missing values in LEVERAGE. Replace with 1:

In [36]:
in_trades_df['LEVERAGE'] = in_trades_df['LEVERAGE'].fillna(1)

In [37]:
tickers_from_trades = in_trades_df['TICKER']
tickers_from_trades.drop_duplicates(inplace=True)
len(tickers_from_trades)

15

## US-SHARES-ISN.csv

In [38]:
in_stocks_df = pd.read_csv(data_directory + 'US-SHARES-ISN.csv')
in_stocks_df.head()

,Name,Ticker,SEDOL,ISIN,Exchange
0,1-800-FLOWERS.COM INC. CL A,FLWS,2444123,US68243Q1067,NASDAQ
1,1ST SOURCE CORP.,SRCE,2341848,US3369011032,NASDAQ
2,2U INC,TWOU,BKWBZZ0,US90214J1016,NASDAQ
3,3D SYSTEMS CORP,DDD,2889768,US88554D2053,New York Stock Exchange Inc.
4,3M CO,MMM,2595708,US88579Y1010,New York Stock Exchange Inc.


In [39]:
tickers_from_shares = in_stocks_df['Ticker']
tickers_from_shares.drop_duplicates(inplace=True)
len(tickers_from_shares)

770

## AllowedProducts.xlsx

In [40]:
in_allowed_df = pd.read_excel(data_directory + 'AllowedProducts.xlsx')
in_allowed_df.head()

,NAME,TICKER,SEDOL,ISIN,EXCHANGE
0,AMGEN INC,AMGN,2023607,US0311621009,NASDAQ
1,COCA-COLA,KO,2206657,US1912161007,New York Stock Exchange Inc.
2,DOW CHEMICAL,DOW,2278719,US2605431038,New York Stock Exchange Inc.
3,DUKE ENERGY CORP,DUK,B7VD3F2,US26441C2044,New York Stock Exchange Inc.
4,JOHNSON & JOHNSON,JNJ,2475833,US4781601046,New York Stock Exchange Inc.


In [41]:
tickers_from_allowed = in_allowed_df['TICKER']

## LASTPRICES.xlsx

In [42]:
in_lastprices_df = pd.read_excel(data_directory + 'LASTPRICES.xlsx',usecols='B:C')
in_lastprices_df.head()

,TICKER,PRICE
0,AMGN,216.94
1,KO,48.06
2,DOW,45.49
3,DUK,92.11
4,JNJ,137.11


In [43]:
tickers_from_lastprices = in_lastprices_df['TICKER']

## Build Stock table

based on tickers referenced in data

In [69]:
all_tickers = set(tickers_from_trades).union(tickers_from_shares, tickers_from_allowed, tickers_from_lastprices)
len(all_tickers)

785

Get name for tickers (this step can take a while. restart kernel and set global variable mock = True to use mock connection)  
Invalid tickers will have value None

In [73]:
cache_filename = data_directory + 'ticker_cache.pkl'
if path.exists(cache_filename):
    with open(cache_filename, 'rb') as cache_file:
        tickers = pickle.load(cache_file)
else:
    tickers = {}
tickers_to_process = [t for t in all_tickers if t not in tickers.keys()]
ticker_names = track_progress(get_name, tickers_to_process)
tickers.update(dict(zip(tickers_to_process, ticker_names)))
with open(cache_filename, 'wb') as cache_file:
    pickle.dump(tickers, cache_file)
print(f'proccessed a total of {len(tickers)} out of {len(tickers) + len(tickers_to_process) - len(ticker_names)}')

proccessed a total of 785 out of 785


Identify invalid tickers:

In [74]:
for t, n in tickers.items():
    if not n:
        print(f'Invalid ticker: {t}')

Invalid ticker: YHOO
Invalid ticker: FCEA


Remove invalid tickers:

In [75]:
cleaned_tickers = { t:n for t,n in tickers.items() if n }

Create GUIDs for tickers:

In [76]:
tickers_with_id = {t:[str(uuid.UUID(int=random.getrandbits(128))), n] for t,n in cleaned_tickers.items()}

Create table:

In [77]:
stocks_df = pd.DataFrame.from_dict(tickers_with_id, columns=['fi_id', 'name'], orient='index')
stocks_df['ticker'] = stocks_df.index
stocks_df.head()

,fi_id,name,ticker
HFWA,78ec359d-4ab6-cf10-f3fc-1b5f20a49b71,Heritage Financial,HFWA
FAX,7ac76496-e16c-f6a6-44b8-2c3102e71cd2,Aberdeen Asia-Pacific,FAX
HNI,ecdaa2bb-7de3-95a7-a311-48e4fba24f2a,Hon Industries Inc,HNI
NTLA,073d08bc-5128-71fb-efbf-693438d5c5d2,Intellia Thera CS,NTLA
MBVT,4c9990ef-df10-326f-b5e3-4eee9a08a9c3,Merchants Bancshares,MBVT


In [78]:
stocks_df.to_csv(f'{data_directory}{prefix}stock.csv', header=False, index=False)

# Option

Data is generated to demonstrate how that table works. This data is completely made up.

In [79]:
def generate_option():
    fi_id = str(uuid.UUID(int=random.getrandbits(128)))
    underlying_stock = stocks_df['fi_id'][random.randrange(len(stocks_df))]
    option_type = 'B' if random.randrange(2) else 'S'
    style_code = random.randrange(len(option_styles_df))
    strike_price = random.randrange(100, 200)
    # maturity format YYYY-MM-DD hh:mm:ss
    maturity = f'2022-{str(random.randrange(5, 13)).zfill(2)}-{str(random.randrange(1, 30)).zfill(2)} 18:00:00'
    return [fi_id, underlying_stock, option_type, style_code, strike_price, maturity]

generate some options and store in table:

In [80]:
options = [generate_option() for _ in range(50)]
options_df = pd.DataFrame(options, columns=['fi_id', 'underlying_stock', 'option_type', 'style_code', 'strike_price', 'maturity'])
options_df.head()

,fi_id,underlying_stock,option_type,style_code,strike_price,maturity
0,531cfd40-c1c0-463f-4502-0333177328dd,eee7d474-0a06-875f-0d4a-8219f66355da,B,1,162,2022-11-22 18:00:00
1,1333086c-0cf8-cdee-bd35-1b1256a59adb,1fcdc9b4-32a7-82ea-22c5-0ff652de5576,S,0,122,2022-12-16 18:00:00
2,24cfc6b0-eda9-d743-34b8-afea4979d363,2d2d4352-1939-9680-37d8-d7b3c1b098f6,S,0,128,2022-08-01 18:00:00
3,b6005d73-fea3-6381-47a2-24ee649e3b6c,e0ac0bdf-2143-e655-2b04-9efd84625eeb,B,1,101,2022-06-09 18:00:00
4,8168819b-cccb-f7fb-4614-3920306b67aa,d8d49a8e-e540-6eae-7a1a-ee6f1a396bea,S,2,137,2022-08-02 18:00:00


In [81]:
options_df.to_csv(f'{data_directory}{prefix}option.csv', header=False, index=False)

# financial_instrument

Create financial instruments table. Current value is completely made up

Build table from stocks and options

In [82]:
def get_exchange(ticker):
    shares_result = in_stocks_df[in_stocks_df['Ticker'] == ticker]['Exchange']
    if len(shares_result):
        return shares_result.iat[0]
    allowed_result = in_allowed_df[in_allowed_df['TICKER'] == ticker]['EXCHANGE']
    if len(allowed_result):
        return allowed_resul.iat[0]
    online_result = (ticker)
    if online_result:
        return online_result
    else:
        print(f'No exchange found for: {ticker}')
        

In [83]:
get_exchange('TSLA')

'NASDAQ'

In [84]:
ticker_exchange_map = { t:get_exchange(t) for t in tickers.keys() }

No exchange found for:

In [85]:
for t,ex in ticker_exchange_map.items():
    if not ex:
        print(t)

Add stocks to financial instruments:

In [86]:
financial_instruments = []
for t, (fi_id, n) in tickers_with_id.items():
    value = random.randint(40, 250)
    financial_instruments.append([fi_id, 0, value, 'USD', ticker_exchange_map[t]])
financial_instruments[:2]

[['78ec359d-4ab6-cf10-f3fc-1b5f20a49b71', 0, 122, 'USD', 'NASDAQ'],
 ['7ac76496-e16c-f6a6-44b8-2c3102e71cd2', 0, 88, 'USD', 'FAX']]

Add options to financial instruments:

In [87]:
for o in options:
    fi_id, underlying_stock, option_type, style_code, strike_price, maturity = o
    value = random.randint(40, 250)
    # set exchange to exchange of underlying stock
    underlying_ticker = stocks_df[stocks_df['fi_id'] == underlying_stock]['ticker'].iat[0]
    # use USD as ccy for all. It is deembed reasonable to do that as the list of stocks was labeled as US shares
    financial_instruments.append([fi_id, 1, value, 'USD', ticker_exchange_map[underlying_ticker]])
financial_instruments[-2:]

[['ffb77a50-7b07-1490-ea6e-ecf22f5c45a2', 1, 100, 'USD', 'NASDAQ'],
 ['c2e43089-a6f8-f016-5c57-ac7c01cf95fe',
  1,
  219,
  'USD',
  'New York Stock Exchange Inc.']]

In [88]:
fi_df = pd.DataFrame(financial_instruments, columns=['fi_id', 'type_code', 'current_value', 'currency_iso', 'exchange'])
fi_df.head()

,fi_id,type_code,current_value,currency_iso,exchange
0,78ec359d-4ab6-cf10-f3fc-1b5f20a49b71,0,122,USD,NASDAQ
1,7ac76496-e16c-f6a6-44b8-2c3102e71cd2,0,88,USD,FAX
2,ecdaa2bb-7de3-95a7-a311-48e4fba24f2a,0,198,USD,New York Stock Exchange Inc.
3,073d08bc-5128-71fb-efbf-693438d5c5d2,0,123,USD,NASDAQ
4,4c9990ef-df10-326f-b5e3-4eee9a08a9c3,0,133,USD,NASDAQ


In [89]:
fi_df.to_csv(f'{data_directory}{prefix}financial_instrument.csv', header=False, index=False)

# Trades

Add trades from first_price and generate missing data randomly

In [90]:
in_trades_df.head()

,TICKER,ACTION,SHARES,LEVERAGE,PRICE
0,META,B,20001,1.0,912.11
1,DHY,B,401,2.1,2.14
2,FAX,B,2440,3.3,3.28
3,DSU,S,99393,4.0,92.11
4,XOM,B,266,1.0,213.22


In [91]:
trades = []
for i in range(len(in_trades_df)):
    row = in_trades_df.iloc[i]
    ticker, action, shares, leverage, price = row
    trades.append([
        str(uuid.UUID(int=random.getrandbits(128)))                                            #trade_id (generate new id)
        ,trader_df['emp_id'].iat[random.randrange(len(trader_df))] #emp_id (choose random)
        ,stocks_df[stocks_df['ticker'] == ticker]['fi_id'].iat[0]    #fi_id (lookup fi_id from ticker)
        ,cp_df['guid'].iat[random.randrange(len(cp_df))]             #counterparty_id (choose random from available)
        ,f'2022-{str(random.randrange(1, 4)).zfill(2)}-{str(random.randrange(1, 29)).zfill(2)} 14:30:00' #trade_dt (generate randomly) format YYYY-MM-DD hh:mm:ss
        ,action                                                      #action
        ,shares                                                      #quantity
        ,leverage                                                    #leverage
        ,price                                                       #price
        ,'USD'                                                       #currency_iso
        ,0                                                           #margin 
        ,0                                                           #fees
        ,"2022-04-14 16:00:00"                                       #insert_dt (will be auto generated by mysql)
        ,None                                                        #amended_dt (optional)
    ])

In [92]:
trades_df = pd.DataFrame(trades, columns=['trade_id', 'emp_id', 'fi_id', 'counterparty_id', 'trade_dt', 'action', 'quantity', 'leverage', 'price', 'currency_iso', 'margin', 'fees', 'insert_dt', 'amended_dt'])
trades_df.head()

,trade_id,emp_id,fi_id,counterparty_id,trade_dt,action,quantity,leverage,price,currency_iso,margin,fees,insert_dt,amended_dt
0,23e7138f-6d89-627e-c09e-a4d383f39efe,00002344,e27a0a32-d318-7f61-1072-521a3a97460c,349beaae-51dc-441f-29f7-87fae229e5c0,2022-03-02 14:30:00,B,20001,1.0,912.11,USD,0,0,2022-04-14 16:00:00,None
1,357c251f-f218-6332-2500-70e1a4fd2540,00005384,8cab51c5-0bce-40b9-c3c3-e4df26aa1598,db4eeeed-a996-b154-f252-7543d2e632fe,2022-03-17 14:30:00,B,401,2.1,2.14,USD,0,0,2022-04-14 16:00:00,None
2,ae5e3dc0-2181-88a8-20db-18e6f1baa004,00009313,7ac76496-e16c-f6a6-44b8-2c3102e71cd2,dcb0b870-58ab-2b61-d90e-bac98b77a593,2022-02-12 14:30:00,B,2440,3.3,3.28,USD,0,0,2022-04-14 16:00:00,None
3,fdefe6b2-ccf2-715e-adff-92ec27497bb4,00002282,4a4ff9d2-2446-4c8a-1518-56ca5e9ac3ab,fa355d27-62af-b68e-2455-72fed0941885,2022-01-20 14:30:00,S,99393,4.0,92.11,USD,0,0,2022-04-14 16:00:00,None
4,abe8f52c-71b9-de2d-c067-816b6866599b,00009736,4b8eee49-232c-fb8b-53d4-746c9600bc77,4113f200-882a-6117-fb6f-d36dcb65ae19,2022-02-21 14:30:00,B,266,1.0,213.22,USD,0,0,2022-04-14 16:00:00,None


In [93]:
trades_df.to_csv(f'{data_directory}{prefix}trades.csv', header=False, index=False, na_rep='NULL')